In [1]:
#r ".\binaries\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [2]:
using BoSSS.Solution.Tecplot;
using ilPSP.Connectors.Matlab;
using static ilPSP.IMatrixExtensions;
using System.IO;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.CompressibleFlowCommon.ShockFinding;
using System.Collections;

In [3]:
string dbPath = "bosss_db_levelSets";
//string dbPath = "bosss_db_shockFinding";
var database=  OpenOrCreateDatabase(dbPath);

Creating database 'bosss_db_levelSets'.


In [4]:
// Select sessions
List<ISessionInfo> sessions = new List<ISessionInfo>();
//sessions.AddRange(database.Sessions.Pick(6, 7, 8));
sessions.AddRange(database.Sessions.Pick(0));

Error: System.ArgumentOutOfRangeException: Index was out of range. Must be non-negative and less than the size of the collection. (Parameter 'index')
   at System.Collections.Generic.List`1.get_Item(Int32 index)
   at System.Linq.Enumerable.ElementAt[TSource](IEnumerable`1 source, Int32 index)
   at BoSSS.Foundation.IO.IEnumerableExtensions.Pick[T](IEnumerable`1 entities, Int32 index) in C:\experimental\public\src\L4-application\BoSSSpad\IEnumerableExtensions.cs:line 131
   at Submission#5.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [5]:
// Select seeding setup
SeedingSetup seeding = SeedingSetup.av;
//SeedingSetup seeding = SeedingSetup.av3x3;
//SeedingSetup seeding = SeedingSetup.everywhere;

In [6]:
// Select the maximum number of iterations
int maxNumOfIterations = 100; 

In [8]:
// Create Direcotories
//string mainPath      = @"C:\tmp\ProductionRuns_Seeding_" + seeding.ToString() +  @"\";
//string mainPath        = @"C:\tmp\ProductionRuns_Seeding_" + seeding.ToString() +  @"_"+ maxNumOfIterations + @"\"; 
string mainPath = @"C:\tmp\XDGFields_" + seeding.ToString() +  @"_" + maxNumOfIterations.ToString() + @"\"; 

string[] sessionPathsRecon = ShockFindingExtensions.CreateDirectories(mainPath, @"LevelSetReconstruction", sessions);
ShockFindingExtensions.EmptyDirectories(mainPath, @"LevelSetReconstruction", sessions);

string[] sessionPathsInflec = ShockFindingExtensions.CreateDirectories(mainPath, @"InflectionPoints", sessions);


# Find the inflection points (takes time: about 400s for 1000 max Number of Iterations)

In [9]:
InflectionPointFinder finder = new InflectionPointFinder(sessionPathsInflec[0], session.Timesteps.Last());
    finder.FindPoints(seeding: seeding, patchRecoveryGradient: true, patchRecoveryHessian: true, eliminateNonConverged: true, maxNumOfIterations: maxNumOfIterations);
    //finder.Plot(plotDGFields: true, plotSeedingsPoints: true, plotInflectionsPoints: true, plotCurves: false, plotStartEndPairs: false);


Error: (1,81): error CS0103: The name 'session' does not exist in the current context

In [10]:
finder.Results.SaveResults(finder.SessionPath);
finder.ResultsExtended.SaveResults(finder.SessionPath);

Error: (1,1): error CS0103: The name 'finder' does not exist in the current context
(1,28): error CS0103: The name 'finder' does not exist in the current context
(2,1): error CS0103: The name 'finder' does not exist in the current context
(2,36): error CS0103: The name 'finder' does not exist in the current context

In [11]:
MultidimensionalArray results         = ShockFindingExtensions.LoadResults(sessionPathsInflec[0]);
MultidimensionalArray resultsExtended = ShockFindingExtensions.LoadResultsExtended(sessionPathsInflec[0]);

LevelSetReconstruction lsr = new LevelSetReconstruction(sessionPathsRecon[0], session, results, resultsExtended);
resultsExtended.SaveToTextFile("convergedCandidatePoints.txt");
clusteringZero.SaveToTextFile("clusteringZero.txt");
clusterZero.SaveToTextFile("clusterOne.txt");
clusteringTwo.SaveToTextFile("clusteringTwo.txt");
clusteringThree.SaveToTextFile("clusteringThree.txt");

Error: (4,79): error CS0103: The name 'session' does not exist in the current context
(6,1): error CS0103: The name 'clusteringZero' does not exist in the current context
(7,1): error CS0103: The name 'clusterZero' does not exist in the current context
(8,1): error CS0103: The name 'clusteringTwo' does not exist in the current context
(9,1): error CS0103: The name 'clusteringThree' does not exist in the current context

In [12]:

// Clustering zero (density)
    MultidimensionalArray clusteringZero = lsr.CreateClustering_Density(3, new double[] {1.0, 2.7, 5.0});
    //lsr.SaveClusteringToTextFile(clusteringZero);
    MultidimensionalArray clusterZero = lsr.SelectCluster(clusteringZero, clusterToSelect: 1);
    
    // Clustering one (artificial viscosity)
    //MultidimensionalArray clusteringOne = lsr.CreateClustering_AV(clusterZero, 2, new double[] {0.0, 0.0});
    //lsr.SaveClusteringToTextFile(clusteringOne);
    //MultidimensionalArray clusterOne = lsr.SelectCluster(clusteringOne, clusterToSelect: 0);
    
    // Clustering two (eliminate boundary cells)
    MultidimensionalArray clusteringTwo = lsr.CreateClustering_Boundary(clusterZero);
    //lsr.SaveClusteringToTextFile(clusteringTwo);
    
    // Clustering three (eliminate cells close to the blunt body)
    MultidimensionalArray clusteringThree = lsr.CreateClustering_DeletePointsAtGeometry(clusteringTwo);
    //lsr.SaveClusteringToTextFile(clusteringThree);
    // Reconstruct level set field
    lsr.ReconstructLevelSet(patchRecovery: true, continuous: true);
    
    // Plot level set fields
    //lsr.PlotFields();


Error: (3,44): error CS0103: The name 'lsr' does not exist in the current context
(5,41): error CS0103: The name 'lsr' does not exist in the current context
(13,43): error CS0103: The name 'lsr' does not exist in the current context
(17,45): error CS0103: The name 'lsr' does not exist in the current context
(20,5): error CS0103: The name 'lsr' does not exist in the current context

In [13]:
    
    // Save level set and conservative fields
    var exportDb = OpenOrCreateDatabase(lsr.SessionPath + @"bosss_db_levelSetReconstruction");

    DGField[] conservativeFields = new DGField[4];
    conservativeFields[0] = session.Timesteps.Last().Fields.Find("rho");
    conservativeFields[1] = session.Timesteps.Last().Fields.Find("m0");
    conservativeFields[2] = session.Timesteps.Last().Fields.Find("m1");
    conservativeFields[3] = session.Timesteps.Last().Fields.Find("rhoE");


Error: (3,41): error CS0103: The name 'lsr' does not exist in the current context
(6,29): error CS0103: The name 'session' does not exist in the current context
(7,29): error CS0103: The name 'session' does not exist in the current context
(8,29): error CS0103: The name 'session' does not exist in the current context
(9,29): error CS0103: The name 'session' does not exist in the current context

In [14]:
    
    DGField[] levelSetFields = lsr.LevelSetFields.ToArray();


Error: (2,32): error CS0103: The name 'lsr' does not exist in the current context

In [15]:

    DGField[] allFields = new DGField[levelSetFields.Length + conservativeFields.Length];


Error: (2,39): error CS0103: The name 'levelSetFields' does not exist in the current context
(2,63): error CS0103: The name 'conservativeFields' does not exist in the current context

In [16]:
    levelSetFields.CopyTo(allFields, 0);


Error: (1,5): error CS0103: The name 'levelSetFields' does not exist in the current context
(1,27): error CS0103: The name 'allFields' does not exist in the current context

In [17]:
    conservativeFields.CopyTo(allFields, levelSetFields.Length);


Error: (1,5): error CS0103: The name 'conservativeFields' does not exist in the current context
(1,31): error CS0103: The name 'allFields' does not exist in the current context
(1,42): error CS0103: The name 'levelSetFields' does not exist in the current context

In [18]:
allFields

Error: (1,1): error CS0103: The name 'allFields' does not exist in the current context

In [19]:
var texplot = new Tecplot(allFields.Pick(3).Basis.GridDat, 2);
texplot.PlotFields( "BS_"+ 0,0.0,allFields);
    

Error: (1,27): error CS0103: The name 'allFields' does not exist in the current context
(2,34): error CS0103: The name 'allFields' does not exist in the current context

In [20]:
    
    exportDb.SaveGridAndTimestep(allFields, lsr.Session.ProjectName, lsr.Session.Name);



Error: (2,5): error CS0103: The name 'exportDb' does not exist in the current context
(2,34): error CS0103: The name 'allFields' does not exist in the current context
(2,45): error CS0103: The name 'lsr' does not exist in the current context
(2,70): error CS0103: The name 'lsr' does not exist in the current context

In [21]:
count


Error: (1,1): error CS0103: The name 'count' does not exist in the current context

In [22]:
Console.WriteLine("\n### LevelSetReconstruction.bws finished ###");


### LevelSetReconstruction.bws finished ###


In [25]:
// Test data
//MultidimensionalArray input = MultidimensionalArray.Create(3, 1, 5);
//input[0, 0, 0] = 0;
//input[0, 0, 1] = 0.1;
//input[0, 0, 2] = 0.2;
//input[0, 0, 3] = 0.3;
//input[0, 0, 4] = 0.4;

//input[1, 0, 0] = 1.0;
//input[1, 0, 1] = 1.1;
//input[1, 0, 2] = 1.2;
//input[1, 0, 3] = 1.3;
//input[1, 0, 4] = 1.4;

//input[2, 0, 0] = 2.0;
//input[2, 0, 1] = 2.1;
//input[2, 0, 2] = 2.2;
//input[2, 0, 3] = 2.3;
//input[2, 0, 4] = 2.4;

//MultidimensionalArray inputExtended = MultidimensionalArray.Create(3, 3);
//inputExtended[0, 0] = 1;
//inputExtended[0, 1] = 1;
//inputExtended[0, 2] = 0;

//inputExtended[1, 0] = 1;
//inputExtended[1, 1] = -1;
//inputExtended[1, 2] = 1;

//inputExtended[2, 0] = 1;
//inputExtended[2, 1] = 1;
//inputExtended[2, 2] = 2;